# Session 03 - Types, type inference and stability

In [1]:
using Pkg;
Pkg.activate(".");
Pkg.add("Plots");
Pkg.add("BenchmarkTools");
Pkg.update()
Pkg.status()

  Activating environment at `~/Documents/GitHub/Phys215-202122-1/03-Types/Project.toml`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Phys215-202122-1/03-Types/Project.toml`
  No Changes to `~/Documents/GitHub/Phys215-202122-1/03-Types/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/GitHub/Phys215-202122-1/03-Types/Project.toml`
  No Changes to `~/Documents/GitHub/Phys215-202122-1/03-Types/Manifest.toml`
    Updating registry at `~/.julia/registries/General`
  No Changes to `~/Documents/GitHub/Phys215-202122-1/03-Types/Project.toml`
  No Changes to `~/Documents/GitHub/Phys215-202122-1/03-Types/Manifest.toml`


      Status `~/Documents/GitHub/Phys215-202122-1/03-Types/Project.toml`
  [6e4b80f9] BenchmarkTools v1.2.0
  [91a5bcdd] Plots v1.22.4


# Session 3 OKR

**OBJECTIVE**: Demonstrate the dynamic programming features of Julia
 - [ ] **KR1:** Shown or demonstrated the hierarchy of Julia's type hierarchy using the command `subtypes()`. 
    Start from `Number` and use `subtypes()` to explore from _abstract types_ down to _specific types_. 
    Use `supertype()` to determine the _parent_ abstract type.
 - [ ] **KR2:** Implemented and used at least one own composite type via `struct`.
   Generate two more versions that are mutable type and type-parametrized of the custom-built type.
 - [ ] **KR3:** Demonstrated type inference in Julia.
   [Generator expressions](https://docs.julialang.org/en/v1/manual/arrays/#Generator-Expressions) may be used for this.
 - [ ] **KR4:** Created a function with inherent type-*instability*.
   Create a version of the function with fixed *type-instability* issues.
 - [ ] **KR5:** Demonstration of how `@code_warntype` can be useful in detecting *type-instability*.
 - [ ] **KR6:** Demonstration of how `Array`s containing ambiguous/abstract types often results to slow execution of codes.
   The `BenchmarkTools` may be useful in this part.

# Julia type hierarchy

The highest data type in Julia is `Any`.

In [2]:
alltypes = subtypes(Any);
println("There are $(length(alltypes)) types in Julia!")

There are 514 types in Julia!


In [3]:
alltypes

514-element Vector{Any}:
 AbstractArray
 AbstractChannel
 AbstractChar
 AbstractDict
 AbstractDisplay
 AbstractMatch
 AbstractPattern
 AbstractSet
 AbstractString
 Any
 Base.AbstractBroadcasted
 Base.AbstractCartesianIndex
 Base.AbstractCmd
 ⋮
 Tuple
 Type
 TypeVar
 UndefInitializer
 Val
 Vararg
 VecElement
 VersionNumber
 WeakRef
 ZMQ.Context
 ZMQ.Socket
 ZMQ._Message

In [4]:
subtypes(Number)

2-element Vector{Any}:
 Complex
 Real

In [5]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

In [6]:
subtypes(AbstractIrrational)

1-element Vector{Any}:
 Irrational

In [7]:
subtypes(AbstractFloat)

4-element Vector{Any}:
 BigFloat
 Float16
 Float32
 Float64

## Session 3 OKR

**OBJECTIVE**: Demonstrate the dynamic programming features of Julia
 - [x] **KR1:** Shown or demonstrated the hierarchy of Julia's type hierarchy using the command `subtypes()`. 
    Start from `Number` and use `subtypes()` to explore from _abstract types_ down to _specific types_. 
    Use `supertype()` to determine the _parent_ abstract type.
 - [ ] **KR2:** Implemented and used at least one own composite type via `struct`.
   Generate two more versions that are mutable type and type-parametrized of the custom-built type.
 - [ ] **KR3:** Demonstrated type inference in Julia.
   Generator expressions may be used for this.
 - [ ] **KR4:** Created a function with inherent type-*instability*.
   Create a version of the function with fixed *type-instability* issues.
 - [ ] **KR5:** Demonstration of how `@code_warntype` can be useful in detecting *type-instability*.
 - [ ] **KR6:** Demonstration of how `Array`s containing ambiguous/abstract types often results to slow execution of codes.
   The `BenchmarkTools` may be useful in this part.

# Creating customized Type via `struct`

The keyword `struct` instructs Julia to create new data type structures similar to the behavior in `c/c++`.

In [8]:
?struct

search: struct isstructtype mutable struct unsafe_trunc



```
struct
```

The most commonly used kind of type in Julia is a struct, specified as a name and a set of fields.

```julia
struct Point
    x
    y
end
```

Fields can have type restrictions, which may be parameterized:

```julia
struct Point{X}
    x::X
    y::Float64
end
```

A struct can also declare an abstract super type via `<:` syntax:

```julia
struct Point <: AbstractPoint
    x
    y
end
```

`struct`s are immutable by default; an instance of one of these types cannot be modified after construction. Use [`mutable struct`](@ref) instead to declare a type whose instances can be modified.

See the manual section on [Composite Types](@ref) for more details, such as how to define constructors.


## A point mass `PointMass`

In [9]:
struct PointMass
    r::Vector
    m::Real
end

!!! warning Without parentheses, the variable name is a reference name only.

`PointMass` and `PointMass()` are variable names of different natures.

In [10]:
pm = PointMass # !!! assigns an alias
println("typeof(pm) = $(typeof(pm)).")
println("typeof(PointMass) = $(typeof(PointMass)).")

typeof(pm) = DataType.
typeof(PointMass) = DataType.


## Instantating `struct`s

The default constructor for a `struct` is simply an enumeration of the initial values for each `struct` variables according to how it appears.

In [11]:
pm = PointMass([0.0,0.0], 1.0);
println("typeof(pm) = $(typeof(pm)).");
println("pm = $(pm)");

pm

typeof(pm) = PointMass.
pm = PointMass([0.0, 0.0], 1.0)


PointMass([0.0, 0.0], 1.0)

Other constructors can be enumerated by defining new functions of the same name as the `struct`.
For example, customized default constructor `PointMass()` for the `struct` above.

In [12]:
PointMass() = PointMass([0.0, 0.0], 1.0) #default position is at Origin, mass is unit mass of 1.0

PointMass

In [13]:
pm = PointMass()

PointMass([0.0, 0.0], 1.0)

In [14]:
pm.r

2-element Vector{Float64}:
 0.0
 0.0

In [15]:
pm.m

1.0

### Trivia

When `struct`s are composite of fundamental types **and** since they are immutable by nature, such `struct`s instantiated are considered `const` and are saved in the same **exact** memory location!

In [16]:
struct Pixel
    x::Int64
    y::Int64
    color::Char
end

Pixel() = Pixel(1,1,'b')

Pixel

In [17]:
px = Pixel(1,1,'b')

Pixel(1, 1, 'b')

In [18]:
px == Pixel()

true

In [19]:
px === Pixel()

true

In [20]:
println(pm);
println(PointMass());

PointMass([0.0, 0.0], 1.0)
PointMass([0.0, 0.0], 1.0)


In [21]:
pm == PointMass()

false

In [22]:
pm1 = PointMass();
pm == pm1

false

**Why!?**

### Back to regular programming..
Now, we cannot assign new values for the references within the `PointMass` objects.

In [23]:
pm.m = 2.0

LoadError: setfield! immutable struct of type PointMass cannot be changed

### Reference value cannot be changed

Julia does not allow changes in reference location.

```
memory -->       [ ... |0x123**|0x124**|0x125**|0x126**|0x127**| ... ]
label/reference->[ ... | pm.r  | pm.m  |pm.r[1]|pm.r[2]| ...         ]
content/value -->[ ... |0x125**| 1.0   | 0.0   | 0.0   | ...         ]
```

In [24]:
pm.r = [2.0,3.0]

LoadError: setfield! immutable struct of type PointMass cannot be changed

**BUT** the value(s) pointed to by these references may be changed!

In [25]:
pm.r[:] = [1.0, 2.0]

2-element Vector{Float64}:
 1.0
 2.0

In [26]:
pm

PointMass([1.0, 2.0], 1.0)

Values at locations pointed by immutable reference can be modified as long as memory size is the same..

In [27]:
pm.r[:] = [1.0, 2.0, 3.0]

LoadError: DimensionMismatch("tried to assign 3 elements to 2 destinations")

## Creating `mutable struct`s

In [28]:
mutable struct MPointMass
    r::Vector
    m::Real
end

MPointMass() = MPointMass([0.0, 0.0], 1.0)

MPointMass

In [29]:
mpm = MPointMass([1.0, 2.0], 3.0)

MPointMass([1.0, 2.0], 3.0)

In [30]:
mpm.r

2-element Vector{Float64}:
 1.0
 2.0

In [31]:
mpm.r = rand(4)

4-element Vector{Float64}:
 0.20574778262567572
 0.029830601444319704
 0.6243455420826749
 0.4361453562011368

### Downsides:

- ❗ Danger in the change in basic properties of the object!
- Such assignments are therefore rendered `unsafe_` and all operations and functions involved must be labelled as such.

### Notes:
 - ✋ The `supertype` (abstract) cannot be changed even in `mutable struct`s.
 - ✋ UNLESS a conversion from one supertype to another is customized.
 
❗Unstable type hierarchy

The type hierarchy must be preserved.
 1. Mathematically self-consistent hierarchy
 2. Self-consistent operations between types preserved

In [32]:
typeof(mpm.m)

Float64

In [33]:
mpm.m = 2;
typeof(mpm.m)

Int64

In [34]:
mpm.m = complex(1.0,2.0)

LoadError: InexactError: Real(1.0 + 2.0im)

## Parametrized `struct`

**ONLY IF** code universality requires a level of abstraction, it is best to utilize the parametrization of the Types.

👏 JIT will make sure that an appropriate memory space is allocated for each instantiation (**except** for the `mutable struct` of course).

#### Note:
 - ✋ It is always best to keep the memory allocation fixed at JIT.

In [35]:
struct PPointMass{T} #one can also specify possible T such as {T <: Number}
    r::Vector{T}
    m::T # we will assume that the types are same for spatial and mass variables here.
end

PPointMass() = PPointMass{Float64}([0.0,0.0], 1) #defaults to Float64

PPointMass

In [36]:
pm = PPointMass()

PPointMass{Float64}([0.0, 0.0], 1.0)

## Session 3 OKR

**OBJECTIVE**: Demonstrate the dynamic programming features of Julia
 - [x] **KR1:** Shown or demonstrated the hierarchy of Julia's type hierarchy using the command `subtypes()`. 
    Start from `Number` and use `subtypes()` to explore from _abstract types_ down to _specific types_. 
    Use `supertype()` to determine the _parent_ abstract type.
 - [x] **KR2:** Implemented and used at least one own composite type via `struct`.
   Generate two more versions that are mutable type and type-parametrized of the custom-built type.
 - [ ] **KR3:** Demonstrated type inference in Julia.
   Generator expressions may be used for this.
 - [ ] **KR4:** Created a function with inherent type-*instability*.
   Create a version of the function with fixed *type-instability* issues.
 - [ ] **KR5:** Demonstration of how `@code_warntype` can be useful in detecting *type-instability*.
 - [ ] **KR6:** Demonstration of how `Array`s containing ambiguous/abstract types often results to slow execution of codes.
   The `BenchmarkTools` may be useful in this part.

# Type inference

*Simple*: Ability of Julia to use the appropriate memory allocation for a given data.

👌 This is natural in Python and other high-level languages.

In [37]:
[x^2 for x in 1:5]

5-element Vector{Int64}:
  1
  4
  9
 16
 25

In [38]:
[x^2 for x in 1.0:5.0]

5-element Vector{Float64}:
  1.0
  4.0
  9.0
 16.0
 25.0

## Session 3 OKR

**OBJECTIVE**: Demonstrate the dynamic programming features of Julia
 - [x] **KR1:** Shown or demonstrated the hierarchy of Julia's type hierarchy using the command `subtypes()`. 
    Start from `Number` and use `subtypes()` to explore from _abstract types_ down to _specific types_. 
    Use `supertype()` to determine the _parent_ abstract type.
 - [x] **KR2:** Implemented and used at least one own composite type via `struct`.
   Generate two more versions that are mutable type and type-parametrized of the custom-built type.
 - [x] **KR3:** Demonstrated type inference in Julia.
   Generator expressions may be used for this.
 - [ ] **KR4:** Created a function with inherent type-*instability*.
   Create a version of the function with fixed *type-instability* issues.
 - [ ] **KR5:** Demonstration of how `@code_warntype` can be useful in detecting *type-instability*.
 - [ ] **KR6:** Demonstration of how `Array`s containing ambiguous/abstract types often results to slow execution of codes.
   The `BenchmarkTools` may be useful in this part.

# Type stability

 **Type stability** := Consistency in the mapping of input to output types so that as long as the input is taken within its type set, the output will remain within the output type set.


Type stability of a given code set is **ALWAYS** dependent on the programmer.

It is therefore important ---natural in physicists--- that type hierarchy and type conversion be as natural as possible.

## The ramp function

The ramp function is defined as the integral of the Heaviside function and is defined as
$$
R(x) = \left\{ \begin{matrix}
        0,   & x \leq 0 \\
        x,   & x > 0
        \end{matrix} \right.
$$

In [39]:
R(x) = x < 0 ? 0 : x

R (generic function with 1 method)

In [40]:
R(-2)

0

In [41]:
R(-2.0)

0

In [42]:
R(2)

2

In [43]:
R(2.0)

2.0

In [44]:
println("typeof(R(-2)) = $(typeof(R(-2))).")
println("typeof(R(2)) = $(typeof(R(2))).")

println("typeof(R(-2.0)) = $(typeof(R(-2.0))).")
println("typeof(R(2.0)) = $(typeof(R(2.0))).")

typeof(R(-2)) = Int64.
typeof(R(2)) = Int64.
typeof(R(-2.0)) = Int64.
typeof(R(2.0)) = Float64.


In [45]:
@code_warntype R(2)

Variables
  #self#::Core.Const(R)
  x::Int64

Body::Int64
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x


In [46]:
@code_warntype R(2.0)

Variables
  #self#::Core.Const(R)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return 0
3 ─      return x


## Fixing type instability issue

 - Use inherent functions for known constants: `zero(var)`, `one(var)`
 - Use type information : slower and not type stable

In [47]:
x = 1.0
n = 1

1

In [48]:
one(x)

1.0

In [49]:
one(n)

1

In [50]:
R₀(x) = x < 0 ? zero(x) : x

R₀ (generic function with 1 method)

In [51]:
@code_warntype R₀(-2.0)

Variables
  #self#::Core.Const(R₀)
  x::Float64

Body::Float64
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.zero(x)::Core.Const(0.0)
└──      return %3
3 ─      return x


In [52]:
using BenchmarkTools

In [53]:
@benchmark for _ in 1:100_000 R(-2.0) end

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.073 ns … 60.814 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.141 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.195 ns ±  1.357 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                  ▁            ▂██▄▅           ▂▂  ▁          
  ▂▄▅▄▃▃▃▂▂▂▂▁▁▂▃▅█▅▅▄▃▃▃▂▂▂▁▂▃██████▄▃▃▂▂▂▁▂▂▅██▆▆█▄▃▃▂▂▂▁▂ ▃
  1.07 ns        Histogram: frequency by time         1.2 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [54]:
@benchmark for _ in 1:100_000 R₀(-2.0) end

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.164 ns … 69.530 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.203 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.242 ns ±  0.862 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█       ▃                                                  
  ███▂▇█▃▂▅█▄▂▁▆▄▃▂▁▁▁▁▁▁▃▃▂▂▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▃ ▃
  1.16 ns        Histogram: frequency by time        1.65 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

**Note:** Julia does try to optimize.

## Session 3 OKR

**OBJECTIVE**: Demonstrate the dynamic programming features of Julia
 - [x] **KR1:** Shown or demonstrated the hierarchy of Julia's type hierarchy using the command `subtypes()`. 
    Start from `Number` and use `subtypes()` to explore from _abstract types_ down to _specific types_. 
    Use `supertype()` to determine the _parent_ abstract type.
 - [x] **KR2:** Implemented and used at least one own composite type via `struct`.
   Generate two more versions that are mutable type and type-parametrized of the custom-built type.
 - [x] **KR3:** Demonstrated type inference in Julia.
   Generator expressions may be used for this.
 - [x] **KR4:** Created a function with inherent type-*instability*.
   Create a version of the function with fixed *type-instability* issues.
 - [x] **KR5:** Demonstration of how `@code_warntype` can be useful in detecting *type-instability*.
 - [ ] **KR6:** Demonstration of how `Array`s containing ambiguous/abstract types often results to slow execution of codes.
   The `BenchmarkTools` may be useful in this part.

# Side: Clean codes => less surprises

In [55]:
function sumsqrtn_naive(n)
    ret = 0
    for x in 1:n
        ret = ret + sqrt(x)
    end
end

sumsqrtn_naive (generic function with 1 method)

In [56]:
# Cleaner code, sqrt() <: AbstractFloat
function sumsqrtn_clean(n)
    ret = 0.0
    for x in 1:n
        ret = ret + sqrt(x)
    end
end

sumsqrtn_clean (generic function with 1 method)

In [57]:
mark1 = @benchmark sumsqrtn_naive(1_000_000)

BenchmarkTools.Trial: 9192 samples with 1 evaluation.
 Range (min … max):  395.613 μs …  1.191 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     538.848 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   542.160 μs ± 62.711 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                      ▁  ▃   ▁          ▂  ▃   █                
  ▂▁▃▂▃▂▂▄▂▅▂▄▅▂▅▄▂█▃▄█▃▃█▃▄▃█▄▃█▅▃▃▇▃▄▃█▃▃█▂▂▂█▂▂▂▇▂▂▂▇▂▂▂▂▂▂ ▃
  396 μs          Histogram: frequency by time          675 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [58]:
mark2 = @benchmark sumsqrtn_clean(1_000_000)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.274 ns … 26.530 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.326 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.344 ns ±  0.362 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

    █                 ▂                ▄                 ▁    
  ▄██▇▄▃▄▃▃▂▂▂▂▁▁▁▂▄▇██▃▄▃▄▃▂▂▂▂▁▁▂▁▂▄▇█▇▅▃▃▄▃▃▂▂▂▁▁▁▂▂▂▄█▇▄ ▃
  1.27 ns        Histogram: frequency by time        1.42 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [59]:
median(mark1.times) / median(mark2.times)

406371.04072398186

In [60]:
@code_warntype sumsqrtn_naive(10)

Variables
  #self#::Core.Const(sumsqrtn_naive)
  n::Int64
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  ret::Union{Float64, Int64}
  x::Int64

Body::Nothing
1 ─       (ret = 0)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = ret::Union{Float64, Int64}
│   %11 = Main.sqrt(x)::Float64
│         (ret = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return nothing


In [61]:
@code_warntype sumsqrtn_clean(10)

Variables
  #self#::Core.Const(sumsqrtn_clean)
  n::Int64
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  ret::Float64
  x::Int64

Body::Nothing
1 ─       (ret = 0.0)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = ret::Float64
│   %11 = Main.sqrt(x)::Float64
│         (ret = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return nothing


In [62]:
function sumsqrtn_clner(n)
    ret = 0.0
    for x in 1:n
        ret = ret + sqrt(1.0*n) ## forces conversion before calling function
    end
end

sumsqrtn_clner (generic function with 1 method)

In [63]:
@code_warntype sumsqrtn_clner(10)

Variables
  #self#::Core.Const(sumsqrtn_clner)
  n::Int64
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  ret::Float64
  x::Int64

Body::Nothing
1 ─       (ret = 0.0)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (x = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = ret::Float64
│   %11 = (1.0 * n)::Float64
│   %12 = Main.sqrt(%11)::Float64
│         (ret = %10 + %12)
│         (@_3 = Base.iterate(%2, %9))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return nothing


In [64]:
mark3 = @benchmark sumsqrtn_clner(1_000_000)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.103 ns … 36.046 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.216 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.239 ns ±  0.711 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

          █      ▃              █▁      ▃▆                    
  ▃█▇▂▂▂▃██▇▃▂▁▃▆█▅▃▂▂▂▅█▇▃▁▂▁▃███▃▂▁▁▃▆██▅▂▂▁▁▂▄▆▄▃▁▁▁▁▂▃▃▄ ▃
  1.1 ns         Histogram: frequency by time        1.38 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [65]:
println("mark1/mark2 = $(median(mark1.times) / median(mark2.times))")
println("mark2/mark3 = $(median(mark2.times) / median(mark3.times))")

mark1/mark2 = 406371.04072398186
mark2/mark3 = 1.0904605263157896


## Using SIMD

SIMD := Single Instruction, Multiple Data

Secret sauce for vectorization.

In [66]:
function sumsqrtn_naive_simd(n)
    ret = 0
    @simd for x in 1:n
        ret = ret + sqrt(n)
    end
end

sumsqrtn_naive_simd (generic function with 1 method)

In [67]:
function sumsqrtn_clean_simd(n)
    ret = 0.0
    @simd for x in 1:n
        ret = ret + sqrt(1.0*n)
    end
end

sumsqrtn_clean_simd (generic function with 1 method)

In [68]:
mark1x = @benchmark sumsqrtn_naive_simd(1_000_000)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  263.753 μs … 823.719 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     287.557 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   294.427 μs ±  31.600 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ █▁ ▇▁▁█▂▂█▂▁▃█▁ ▆▁▂ ▇▂  ▄▁                                  ▃
  █▁██▇███████████████████████████▆▅▇█▆▅▆▅▆▆▅▄▅▄▅▅▅▆▅▄▅▄▆▇▅▄▆▃▆ █
  264 μs        Histogram: log(frequency) by time        424 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [69]:
mark2x = @benchmark sumsqrtn_clean_simd(1_000_000)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.198 ns … 56.008 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.208 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.245 ns ±  0.945 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▃                     ▂▁                                 ▂
  ███▇▆▆▅▃▄▄▃▁▃▁▃▁▃▃▃▃▃▁▁▃██▅▁▁▃▁▁▃▁▁▁▄▃▃▁▁▃▁▁▁▃▁▁▁▁▁▁▁▁▃▆▇▇ █
  1.2 ns       Histogram: log(frequency) by time     1.83 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [70]:
println("mark1/mark1x = $(median(mark1.times) / median(mark1x.times))")
println("mark2/mark2x = $(median(mark2.times) / median(mark2x.times))")

mark1/mark1x = 1.8738856537758666
mark2/mark2x = 1.0976821192052981


### Observations

 - @simd macro is not that effective for already efficient loops
 - @simd macro **may** be more efficient with respect to looping over container values

# JIT array allocation

In [71]:
a = Int64[1,2,3,4,5,6,7,8,9,10];
b = Number[1,2,3,4,5,6,7,8,9,10];

In [72]:
typeof(a)

Vector{Int64} (alias for Array{Int64, 1})

In [73]:
typeof(b)

Vector{Number} (alias for Array{Number, 1})

In [74]:
function sumsqr(x::Array{T}) where T <: Number
    ret = zero(T)
    for i in 1:length(x)
        ret = ret + x[i]*x[1]
    end
    return ret
end

sumsqr (generic function with 1 method)

In [75]:
sumsqr(a)

55

In [76]:
sumsqr(b)

55

In [77]:
typeof(sumsqr(a))

Int64

In [78]:
typeof(sumsqr(b))

Int64

In [79]:
mark4Int64 = @benchmark sumsqr($a)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  4.899 ns … 68.514 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.559 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.627 ns ±  1.966 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █        ▂ █                                                
  █▂▅▂▅▁▅▂▂█▂██▂▇▂▂▃▂▂▁▁▂▁▁▁▁▁▁▂▂▂▃▁▂▂▁▂▂▁▁▂▁▂▂▁▁▁▂▂▁▂▂▁▁▁▂▂ ▃
  4.9 ns         Histogram: frequency by time        8.98 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [80]:
mark4Number = @benchmark sumsqr($b)

BenchmarkTools.Trial: 10000 samples with 294 evaluations.
 Range (min … max):  240.939 ns …  1.057 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     311.760 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   310.956 ns ± 35.814 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

              ▇           ▂   █         ▁                       
  ▁▁▂▁▁▂▂▁▃▇▂▂█▃▃▄█▃▂▅█▂▂▃█▂▂▃█▇▃▂▂▇▃▂▂▂█▄▂▂▁▃▇▂▂▁▁▇▅▂▂▁▁▂▁▁▁▁ ▂
  241 ns          Histogram: frequency by time          392 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [81]:
println("mark4Number/mark4Int64 = $(median(mark4Number.times) / median(mark4Int64.times))")

mark4Number/mark4Int64 = 56.08206585386448


## Session 3 OKR

**OBJECTIVE**: Demonstrate the dynamic programming features of Julia
 - [x] **KR1:** Shown or demonstrated the hierarchy of Julia's type hierarchy using the command `subtypes()`. 
    Start from `Number` and use `subtypes()` to explore from _abstract types_ down to _specific types_. 
    Use `supertype()` to determine the _parent_ abstract type.
 - [x] **KR2:** Implemented and used at least one own composite type via `struct`.
   Generate two more versions that are mutable type and type-parametrized of the custom-built type.
 - [x] **KR3:** Demonstrated type inference in Julia.
   Generator expressions may be used for this.
 - [x] **KR4:** Created a function with inherent type-*instability*.
   Create a version of the function with fixed *type-instability* issues.
 - [x] **KR5:** Demonstration of how `@code_warntype` can be useful in detecting *type-instability*.
 - [x] **KR6:** Demonstration of how `Array`s containing ambiguous/abstract types often results to slow execution of codes.
   The `BenchmarkTools` may be useful in this part.

# Compose concrete things; parametrize when necessary

 - Often computations in physics require a very specific data structure.
 Create concrete typed structures!
 - Sometimes a general type of structure is required by the situation.
 Create parametrized type of structures!

In [82]:
struct Fields
    x
    y
end

In [83]:
struct ConcreteFields
    x::Float64
    y::Float64
end

In [84]:
struct AbstractFields
    x::AbstractFloat
    y::AbstractFloat
end

In [85]:
struct ParametricFields{T <: AbstractFloat}
    x::T
    y::T
end

## Kinetic energy (?)

In [86]:
function KE(F)
    ke = 0.0
    for f in F
        ke = ke + f.x^2 + f.y^2
    end
    return ke
end

KE (generic function with 1 method)

In [87]:
fields_plain = [ Fields(rand(),rand()) for _ in 1:100_000 ];
fields_concr = [ ConcreteFields(rand(),rand()) for _ in 1:100_000 ];
fields_abstr = [ AbstractFields(rand(),rand()) for _ in 1:100_000 ];
fields_param = [ ParametricFields(rand(),rand()) for _ in 1:100_000 ];

In [88]:
mark1 = @benchmark KE(fields_plain);
mark2 = @benchmark KE(fields_concr);
mark3 = @benchmark KE(fields_abstr);
mark4 = @benchmark KE(fields_param);

In [89]:
println("mark1/mark2 = $(median(mark1.times) / median(mark2.times))")
println("mark1/mark3 = $(median(mark1.times) / median(mark3.times))")
println("mark1/mark4 = $(median(mark1.times) / median(mark4.times))")

mark1/mark2 = 29.375054795769117
mark1/mark3 = 1.0617136026505025
mark1/mark4 = 30.38034447003057


## Take aways:

 - Easy to let Julia determine the field types
 - Good to have abstract field types
 - Better to have parametrized field types
 - Best if concrete
 
> Do what is right, not what is easy.

# Fin. [Back.](https://jybantang.github.io/Phys215-202122-1/03-Types/)